In [1]:
!sudo apt-get update
!sudo apt-get install tesseract-ocr poppler-utils
!pip install pytesseract pdf2image Pillow opencv-python-headless numpy pycocotools

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,567 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,631 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1

In [ ]:
!pip install pytesseract tox

In [2]:
import json
import os

In [3]:
os.environ['PATH'] += ':/usr/bin'

In [4]:
import pytesseract
print(pytesseract.get_tesseract_version())

4.1.1


In [ ]:
categories = [
    {'id': 1, 'name': 'Seller_Name'},
    {'id': 2, 'name': 'Buyer_Name'},
    {'id': 3, 'name': 'NTN_Number'},
    # Add more categories as needed
]

coco_annotations = {
    'images': [],
    'annotations': [],
    'categories': categories
}

In [ ]:
from PIL import Image

image = Image.open('/content/invoice_15001_page_1.jpeg')

# Extract text
text = pytesseract.image_to_string(image)
print(text)

@)

GENERAL ORDER SUPPLIERS

SALES TAX INVOICE

Serial No.: 995721

Supplier's Details:

Supplier's Name: BEST DAIRY FEEDS

Address: Medical Director Habib Medical Center Bus/3 Block-4
S.T. Reg. No.: 1070-77-8761-1452-23

NTN: 4220172648109

Buyer's Details:

Buyer's Name: MUHAMMAD SHAFIQUE KHAN

Address: H A-287 Jahangir Road 1 Near Jahangir Road Sindh
S.T. Reg. No.: 2811-2857-8761-413-23

NTN: 3169169-2

9.9. Khan “/eadens reso)

Dealers In: HARDWARE, TEXTILE & PACKING MATERIAL

Lahore
Coll

Date: 13-01-2021

Hoad Office; 5, Qaddafi
Markel, Brandreth Road

Tol, 57652650
(900-1420816
0921-4300942

Amount Amount
SR. NO. | Description Quantity | Rate Excluding ST Sales Tax Including ST

3,936.00
480.00

2,982.00
5,185.00

12,583.00

 

 

Sales Tax: 2,264.94
Net Tax Inclusive Value: 14,847.94

Signature:
Name and Designation:

 

708.48
86.40

536.76
933.30

2,264.94

14,847.94



In [ ]:
from pdf2image import convert_from_path

def convert_pdf_to_images(pdf_path, output_folder):
    images = convert_from_path(pdf_path)
    for idx, img in enumerate(images):
        img.save(os.path.join(output_folder, f'{os.path.basename(pdf_path)}_page_{idx+1}.jpg'), 'JPEG')

In [ ]:
import pytesseract
from PIL import Image

def process_image(image_path):
    img = Image.open(image_path)
    width, height = img.size
    image_id = len(coco_annotations['images']) + 1
    coco_annotations['images'].append({
        'id': image_id,
        'file_name': os.path.basename(image_path),
        'width': width,
        'height': height
    })
    data = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)

    # Example: Detect Seller's Name
    for i in range(len(data['text'])):
        text = data['text'][i].strip()
        if text == 'Seller\'s Name:':
            seller_name_text = data['text'][i+1]
            x = data['left'][i+1]
            y = data['top'][i+1]
            w = data['width'][i+1]
            h = data['height'][i+1]
            coco_annotations['annotations'].append({
                'id': len(coco_annotations['annotations']) + 1,
                'image_id': image_id,
                'category_id': 1,  # Seller_Name
                'bbox': [x, y, w, h],
                'area': w * h,
                'iscrowd': 0
            })
        # Add similar logic for other fields

In [ ]:
data_folder = './'
output_folder = './'
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(data_folder):
    file_path = os.path.join(data_folder, filename)
    if filename.endswith('.pdf'):
        convert_pdf_to_images(file_path, output_folder)
    elif filename.endswith(('.jpg', '.jpeg', '.png')):
        process_image(file_path)

In [ ]:
with open(os.path.join(output_folder, 'annotations.json'), 'w') as f:
    json.dump(coco_annotations, f)



---



In [5]:
import os
import cv2
import pytesseract
import json
import numpy as np

# Set Tesseract executable path if required
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def extract_text_with_boxes(image_path):
    """
    Extracts text and bounding boxes from an image using Tesseract OCR.
    """
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # OCR with bounding boxes
    data = pytesseract.image_to_data(gray, output_type=pytesseract.Output.DICT)
    return data, image

def match_labels_and_create_annotations(data, labels):
    """
    Matches extracted text with labels and creates annotations.
    """
    annotations = []
    for i, text in enumerate(data['text']):
        text = text.strip()
        if text:
            for label in labels:
                if label in text:
                    bbox = [data['left'][i], data['top'][i], data['width'][i], data['height'][i]]
                    annotations.append({"label": label, "text": text, "bbox": bbox})
    return annotations

def convert_to_coco_format(annotations, image_id, image_width, image_height):
    """
    Converts annotations to COCO format.
    """
    coco_annotations = []
    for ann_id, ann in enumerate(annotations):
        x, y, width, height = ann['bbox']
        coco_annotations.append({
            "id": ann_id,
            "image_id": image_id,
            "category_id": 1,  # Assign 1 for all labels initially
            "bbox": [x, y, width, height],
            "area": width * height,
            "segmentation": [],
            "iscrowd": 0
        })
    return coco_annotations

def annotate_images_in_folder(folder_path, labels, output_json_path):
    """
    Annotates images in a folder and saves COCO-format annotations.
    """
    coco_dataset = {
        "images": [],
        "annotations": [],
        "categories": [{"id": 1, "name": "text"}]  # Simplify to a single category
    }

    image_id = 1
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('png', 'jpg', 'jpeg')):
            image_path = os.path.join(folder_path, filename)
            data, image = extract_text_with_boxes(image_path)

            # Annotate labels
            annotations = match_labels_and_create_annotations(data, labels)

            # Image dimensions
            height, width, _ = image.shape

            # Append to COCO dataset
            coco_dataset["images"].append({
                "id": image_id,
                "file_name": filename,
                "width": width,
                "height": height
            })
            coco_dataset["annotations"].extend(
                convert_to_coco_format(annotations, image_id, width, height)
            )
            image_id += 1

    # Save to JSON
    with open(output_json_path, 'w') as f:
        json.dump(coco_dataset, f, indent=4)

# Folder Path and Labels
folder_path = "./"
output_json_path = "annotations_coco.json"
labels = ["Supplier's Name", "Buyer's Name", "NTN", "Sales Tax", "Total"]  # Extend with other labels

# Annotate and save
annotate_images_in_folder(folder_path, labels, output_json_path)
print(f"Annotations saved to {output_json_path}")


Annotations saved to annotations_coco.json
